# Visualizing Data from BACnet SImulator via Python

### 1. Starting

- Download Windows YABE from: https://sourceforge.net/projects/yetanotherbacnetexplorer/
- Once installed, go to Windows Start button > All Programs > Yabe folder and run Yabe (with a magnifying glass logo) and Bacnet.Room.Simulator app.

<img src="Yabe_explorer.png">
<br>
<img src="room_control_simulator.png">
<br>

- Start a jupyter notebook on your computer via anaconda prompt.

reference: 

- code is adapted from: https://github.com/techbeast-org/bacnet-ip/tree/master
- instruction video: https://www.youtube.com/watch?v=TyEXDnjBsD8

### 2. Install the necessary libraries:

In [1]:
!pip install BAC0==21.12.3 bacpypes==0.18.6 colorama==0.4.4

In [1]:
#update a compatible pandas library
!pip uninstall -y pandas
!pip install pandas==1.3.5

  Using cached pandas-1.3.5-cp37-cp37m-win32.whl (8.9 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydexp 1.0.0 requires numpy<=1.21.2,>=1.18.0, but you have numpy 1.21.6 which is incompatible.
pydexp 1.0.0 requires pandas<=1.3.3,>=1.0.0, but you have pandas 1.3.5 which is incompatible.


In [16]:
#Install plotly
!pip install plotly

### 3. Communicate with Yabe and display a live data in a graph.

Make sure to input your own local ip address and specify the device's BACnet/I UDP port number (obtained from Yabe explorer):
- 192.168.1.66 → the device’s IP address
- 58956 → the device’s BACnet/IP UDP port

In [3]:
import BAC0
import time
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Setup BACnet
bacnet = BAC0.lite(ip="192.168.1.66/24", port=47818)

# History buffer
history = {
    "Analog_input0": [],
    "Analog_input1": [],
    "Analog_input2": [],
    "time": []
}

# Read values from BACnet
def read_bacnet_inputs():
    values = {}
    for i in range(3):
        point = f"192.168.1.66:58956 analogInput {i} presentValue"
        try:
            value = float(bacnet.read(point))
        except:
            value = None
        values[f"Analog_input{i}"] = value
    return values

# Update Plotly figure
def plot_live():
    fig = go.Figure()
    for key in ["Analog_input0", "Analog_input1", "Analog_input2"]:
        fig.add_trace(go.Scatter(
            x=history["time"], y=history[key],
            mode='lines+markers', name=key
        ))
    fig.update_layout(
        title="Live BACnet Analog Inputs",
        xaxis_title="Time (s)",
        yaxis_title="Value",
        template="plotly_dark",
        height=500
    )
    clear_output(wait=True)
    display(fig)

# Main loop
def main():
    start = time.time()
    while True:
        values = read_bacnet_inputs()
        now = round(time.time() - start, 1)
        history["time"].append(now)
        for key in ["Analog_input0", "Analog_input1", "Analog_input2"]:
            history[key].append(values.get(key))
        plot_live()
        time.sleep(2)

# Run it
if __name__ == '__main__':
    main()


KeyboardInterrupt: 